### PARTICLE SWARM 

In [151]:
import numpy as np
np.set_printoptions(suppress=True)

In [152]:
%run fuzzy_pakar_csv.ipynb

waktu belajar   : {'kecil': 0, 'sedang': 0.6986301369863014, 'besar': 0.08943089430894309}
lama jawab soal : {'kecil': 0, 'sedang': 0, 'besar': 0.07239424334932403}
nilai           : {'kecil': 1, 'sedang': 0, 'besar': 0}
RULE 1: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal kecil (0.0000) AND nilai kecil (1.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 2: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal kecil (0.0000) AND nilai kecil (0.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 3: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal kecil (0.0000) AND nilai kecil (0.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 4: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal sedang (0.0000) AND nilai sedang (1.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 5: IF waktu_belajar kecil (0.0000) AND lama_jawab_soal sedang (0.0000) AND nilai sedang (0.0000) THEN kelulusan tidak lulus : ALPAPREDIKAT = 0.0000
RULE 6: IF waktu_bel

In [153]:
class Particle:
    def __init__(self, bounds):
        self.bounds = bounds
        self.position = self._generate_position()
        self.velocity = self._generate_velocity()
        self.best_position = np.copy(self.position)
        self.best_score = -float('inf')

    def _generate_position(self):
        position = []
        for min_bound, max_bound, length in self.bounds:
            segment = np.random.uniform(min_bound, max_bound, size=length)
            position.extend(segment)
        return np.array(position)

    def _generate_velocity(self):   
        velocity = []
        for _, _, length in self.bounds:
            segment = np.random.uniform(-1, 1, size=length)
            velocity.extend(segment)
        return np.array(velocity)

    def update_velocity(self, global_best_position, w, c1, c2):
        r1 = np.random.rand(len(self.position))
        r2 = np.random.rand(len(self.position))
        self.velocity = w * self.velocity + c1 * r1 * (self.best_position - self.position) + c2 * r2 * (global_best_position - self.position)

    def update_position(self):
        self.position += self.velocity
        start_idx = 0
        for min_bound, max_bound, length in self.bounds:
            end_idx = start_idx + length
            self.position[start_idx:end_idx] = np.clip(
                self.position[start_idx:end_idx], min_bound, max_bound
            )
            start_idx = end_idx

In [154]:
def objective_function(particle_position, dataset):
    prediksi_benar = 0
    for i in range(len(dataset)):
        waktu_belajar = dataset.loc[i, 'rerata_durasi_akses_materi']
        lama_jawab_soal = dataset.loc[i, 'durasi_berlatih']
        nilai = dataset.loc[i, 'nilai']
        aktual_kelulusan = dataset.loc[i, 'kelulusan']
        # print(particle_position)
        x, y, z = func_fuzzifikasi(waktu_belajar, lama_jawab_soal, nilai, particle_position)
        # print(f"waktu belajar   : {x}")
        # print(f"lama jawab soal : {y}")
        # print(f"nilai           : {z}")
        hasil_inferensi, rule_text = inferensi_mamdani(x, y, z, rules)
        hasil_agregasi = agregasi(hasil_inferensi)
        hasil_defuzzifikasi = defuzzifikasi(hasil_agregasi)
        prediksi_kelulusan = kelulusan(hasil_defuzzifikasi)
        if int(prediksi_kelulusan) == int(aktual_kelulusan):
            prediksi_benar += 1
        else:
            continue
    akurasi = (prediksi_benar / len(dataset)) * 100
    return akurasi

In [155]:
def particle_swarm_optimization(bounds, dataset, num_particles, max_iter, w=0.5, c1=2, c2=2):
    swarm = []
    for _ in range(num_particles):
        swarm.append(Particle(bounds))
    global_best_position = None
    global_best_score = -float('inf')

    for iteration in range(max_iter):
        for particle in swarm:
            # Evaluate fitness
            # print(particle.position)
            # print(len(particle.velocity))
            particle_score = objective_function(particle.position, dataset)
            # print(particle_score)
            if particle_score > particle.best_score:
                particle.best_score = particle_score
                particle.best_position = np.copy(particle.position)

            if particle_score > global_best_score:
                global_best_score = particle_score
                global_best_position = np.copy(particle.position)

        # Update velocity and position
        for particle in swarm:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position()
        print(f"Iteration {iteration+1}/{max_iter}, Best Score: {global_best_score}")

    return global_best_position, global_best_score

In [156]:
bounds = [
    (0, 700, 5),  # Elemen 1-5 dengan batas [0 - 700]
    (0, 7500, 5),  # Elemen 6-10 dengan batas [0, 7500]
    (0, 100, 5)  # Elemen 11 - 15 dengan batas [0, 100]
]

# Run PSO
num_particles = 20
max_iter = 20
best_params, best_score = particle_swarm_optimization(bounds, dataset, num_particles, max_iter)

print("\nOptimal Parameters:")
print(best_params)
print(f"Best Accuracy: {best_score:.4f}")

Iteration 1/20, Best Score: 59.813084112149525
Iteration 2/20, Best Score: 81.30841121495327
Iteration 3/20, Best Score: 81.30841121495327
Iteration 4/20, Best Score: 93.45794392523365
Iteration 5/20, Best Score: 93.45794392523365
Iteration 6/20, Best Score: 93.45794392523365
Iteration 7/20, Best Score: 93.45794392523365
Iteration 8/20, Best Score: 93.45794392523365
Iteration 9/20, Best Score: 93.45794392523365
Iteration 10/20, Best Score: 95.32710280373831
Iteration 11/20, Best Score: 95.32710280373831
Iteration 12/20, Best Score: 95.32710280373831
Iteration 13/20, Best Score: 95.32710280373831
Iteration 14/20, Best Score: 95.32710280373831
Iteration 15/20, Best Score: 95.32710280373831
Iteration 16/20, Best Score: 95.32710280373831
Iteration 17/20, Best Score: 95.32710280373831
Iteration 18/20, Best Score: 95.32710280373831
Iteration 19/20, Best Score: 95.32710280373831
Iteration 20/20, Best Score: 95.32710280373831

Optimal Parameters:
[ 674.72904657  560.70919959  391.22716211    0

In [157]:
print(best_params)

[ 674.72904657  560.70919959  391.22716211    0.          428.65028426
  369.63535933 2665.20799512    0.         6000.79799647    0.
   74.15577387   56.13991107   80.36863089   39.39109139    0.        ]
